In [1]:
import os
import numpy as np
import scipy.io
import nibabel as nib
import sys
sys.path.append("..")
from utils_l import naive_project
from utils_l import math_project
from utils_l import euclidean_distance_coords
from mayavi import mlab
from tqdm import tqdm
import pandas as pd

In [2]:
study_id = "DBS_bT20"

In [3]:
prect = nib.load(os.path.join("../data",
                              study_id,
                              "preop_ct.nii"))
Tmatrix = np.transpose(prect.affine)

In [5]:
# this is how the ideal output from Micah's functon should look like
ground_truth_coords = np.load(os.path.join("../data",
                                      study_id,
                                      "electrode_locs.npy"))
temp_coords = np.hstack( (ground_truth_coords, np.ones( (ground_truth_coords.shape[0],1))))
aligned_coords= (temp_coords @ np.linalg.inv(Tmatrix))[:,0:2]

In [6]:
hull = scipy.io.loadmat(os.path.join("../data",
                                     study_id,
                                     "hull_rh.mat"))
points_hull = np.array(hull['mask_indices'])

In [7]:
coords = aligned_coords
coords = np.hstack( (aligned_coords, np.zeros( (aligned_coords.shape[0],1))))
coords = np.hstack( (coords, np.ones( (coords.shape[0],1))))
coords = coords @ Tmatrix
prediction = np.array(naive_project(coords[:,0:3],points_hull))

In [9]:
coords = aligned_coords
coords = np.hstack( (aligned_coords, np.zeros( (aligned_coords.shape[0],1))))
coords = np.hstack( (coords, np.ones( (coords.shape[0],1))))
coords = coords @ Tmatrix
prediction2 = np.array(math_project(coords[:,0:3],points_hull))
prediction2

C:\Users\VukadinoviM\Documents\BrainReg\hull_projector\..\utils_l\utils.py:229: RuntimeWarning: divide by zero encountered in true_divide
  t = (-b - W@V)/(U@V)


array([[ 49.98447447, -73.83932495,  21.71868932],
       [ 47.38822299, -68.83932495,  28.71869308],
       [ 46.38531003, -62.83932495,  34.71869433],
       [ 45.83985548, -55.83932495,  40.71869433],
       [ 46.3228032 , -47.83932495,  44.71869308],
       [ 45.07279944, -39.83932495,  47.71869683],
       [ 43.46953617, -31.83932495,  50.71870059],
       [ 45.0103007 , -21.83932495,  51.71869558]])

In [2]:
m1 = []
m2 = []

In [3]:
df = pd.read_csv("../split.csv")
df = df[df['split']=='test']

In [ ]:
for study_id in tqdm(df.ids):
    try:
        prect = nib.load(os.path.join("../data",
                                      study_id,
                                      "preop_ct.nii"))
        Tmatrix = np.transpose(prect.affine)

        ground_truth_coords = np.load(os.path.join("../data",
                                              study_id,
                                              "electrode_locs.npy"))
        temp_coords = np.hstack( (ground_truth_coords, np.ones( (ground_truth_coords.shape[0],1))))
        aligned_coords= (temp_coords @ np.linalg.inv(Tmatrix))[:,0:2]

        hull = scipy.io.loadmat(os.path.join("../data",
                                             study_id,
                                             "hull_rh.mat"))
        points_hull = np.array(hull['mask_indices'])

        coords = aligned_coords
        coords = np.hstack( (aligned_coords, np.zeros( (aligned_coords.shape[0],1))))
        coords = np.hstack( (coords, np.ones( (coords.shape[0],1))))
        coords = coords @ Tmatrix
        prediction = np.array(naive_project(coords[:,0:3],points_hull))

        coords = aligned_coords
        coords = np.hstack( (aligned_coords, np.zeros( (aligned_coords.shape[0],1))))
        coords = np.hstack( (coords, np.ones( (coords.shape[0],1))))
        coords = coords @ Tmatrix
        prediction2 = np.array(math_project(coords[:,0:3],points_hull))
        prediction2

        m1.append(euclidean_distance_coords(ground_truth_coords, prediction))
        m2.append(euclidean_distance_coords(ground_truth_coords, prediction2))
    except:
        pass

  0%|                                                                                                                                                                                                                                                    | 0/18 [00:00<?, ?it/s]C:\Users\VukadinoviM\Documents\BrainReg\hull_projector\..\utils_l\utils.py:228: RuntimeWarning: divide by zero encountered in true_divide
  t = (-b - W@V)/(U@V)
 78%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                    | 14/18 [02:32<01:00, 15.10s/it]

In [ ]:
m1

In [ ]:
m2

In [14]:
# mlab.init_notebook()
# fig = mlab.figure(bgcolor=(1, 1, 1), size=(500, 500))
# mlab.points3d(points_hull[:,0],points_hull[:,1], points_hull[:,2], color = (0.2,1,.2), scale_factor=10)
# mlab.points3d(prediction[:,0],prediction[:,1],prediction[:,2], color = (1,0,0), scale_factor=20)
# mlab.points3d(vec[0],vec[1],vec[2], color = (0,0,1), scale_factor=20)

#mlab.points3d(label_ex[:,0],label_ex[:,1],label_ex[:,2], color = (1,0,0), scale_factor=10)